# Obligatorio 2 - Big Data Science

Integrantes del grupo: Martín Brian - Joaquín Martínez

Debajo de cada pregunta o tarea incluya las celdas necesarias para desarrolar la respuesta. Puede usar una o varias celdas de código o mark down (https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook)

Para entregar, renombrar este notebook como "Obligatorio 2 - Apellido1 - Apellido 2 - Apellido 3" con los apellidos de los miembros del grupo. Un solo integrante del grupo debe realizar la entrega. 


1. Cargar los datos del Obligatorio 1, de entrenamiento (.data) y validación (.test) en spark dataframes (distintos). Los nombres de las columnas deben corresponder a los especificados en "Attribute Information".

In [1]:
#from pyspark import SparkContext
import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext.getOrCreate()

spark = SQLContext(sc)

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", FloatType(), True),
    StructField("education", StringType(), True),
    StructField("education-num", FloatType(), True),
    StructField("marital-status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital-gain", FloatType(), True),
    StructField("capital-loss", FloatType(), True),
    StructField("hours-per-week", FloatType(), True),
    StructField("native-country", StringType(), True),
    StructField("salary", StringType(), False)
])

In [3]:
#Leemos los archivos
df_data = spark.read.csv("./adult.data", header=False,schema=schema)
df_test = spark.read.csv("./adult.test", header=True,schema=schema)

In [4]:
df_data.toPandas()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
32557,40,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
32558,58,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
32559,22,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K


In [5]:
df_test.toPandas()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
1,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
2,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
3,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.
4,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419.0,Bachelors,13.0,Divorced,Prof-specialty,Not-in-family,White,Female,0.0,0.0,36.0,United-States,<=50K.
16277,64,?,321403.0,HS-grad,9.0,Widowed,?,Other-relative,Black,Male,0.0,0.0,40.0,United-States,<=50K.
16278,38,Private,374983.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
16279,44,Private,83891.0,Bachelors,13.0,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455.0,0.0,40.0,United-States,<=50K.


In [6]:
from pyspark.sql.functions import *

# Limpiamos puntos en la columna salario
df_test = df_test.withColumn('salary', regexp_replace('salary', '<=50K.', '<=50K'))
df_test = df_test.withColumn('salary', regexp_replace('salary', '>50K.', '>50K'))

In [9]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

# convert to vector column first
vector_col = "corr_features"
string_columns = [
    "workclass_indexed",
    "education_indexed",
    "marital_status_indexed",
    "occupation_indexed",
    "relationship_indexed",
    "race_indexed",
    "sex_indexed",
    "native_country_indexed",
    "salary_indexed"
]
columns = string_columns.extend(["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"])

workclass_indexer = StringIndexer(
        inputCol="workclass", outputCol="workclass_indexed", handleInvalid="keep")
education_indexer = StringIndexer(
        inputCol="education", outputCol="education_indexed", handleInvalid="keep")
marital_status_indexer = StringIndexer(
        inputCol="marital-status", outputCol="marital_status_indexed", handleInvalid="keep")
occupation_indexer = StringIndexer(
        inputCol="occupation", outputCol="occupation_indexed", handleInvalid="keep")
relationship_indexer = StringIndexer(
        inputCol="relationship-status", outputCol="relationship_indexed", handleInvalid="keep")
race_indexer = StringIndexer(
        inputCol="race", outputCol="race_indexed", handleInvalid="keep")
sex_indexer = StringIndexer(
        inputCol="sex", outputCol="sex_indexed", handleInvalid="keep")
native_country_indexer = StringIndexer(
        inputCol="native-country", outputCol="native_country_indexed", handleInvalid="keep")
salary_indexer = StringIndexer(
        inputCol="salary", outputCol="salary_indexed", handleInvalid="keep")

assembler = VectorAssembler(inputCols=columns, outputCol=vector_col)
# df_vector = assembler.transform(df_data).select(vector_col)

age	workclass	fnlwgt	education	education-num	marital-status	occupation	relationship	race	sex	capital-gain	capital-loss	hours-per-week	native-country	salary

pipeline_to_get_corr = Pipeline(stages=[
    workclass_indexer,
    education_indexer,
    occupation_indexer,
    relationship_indexer,
    race_indexer,
    sex_indexer,
    native_country_indexer,
    salary_indexer,
    assembler,
    df_vector
])
# pipeline_to_get_corr.fit(df_data)



# get correlation matrix
# matrix = Correlation.corr(df_vector, vector_col)

NameError: name 'Pipeline' is not defined

2. Seleccionar un conjunto relevante de 5 atributos y crear un Spark Pipeline en el que el estimator sea un DecisionTreeClassifier (https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier). Puede utilizar libremente los transformers/estimators de Spark para realizar ingeniería de atributos (StringIndexer, OneHotEncoding, etc).

3. Hacer el fit del pipeline con los datos de entrenamiento. 


In [5]:
#

4. Graficar la curva ROC utilizando los datos de validación (sin usar el paquete de evluación de Spark pyspark.ml.evaluation)

In [6]:
#

5. Escribir dos funciones train() y predict() que creen el arbol de acuerdo a la metodología vista en clase (utilizando entropía como métrica de homogeneidad de clases).
```
def train(train_dataframe):
    '''
    @return devuelve una estructura de datos que representa el árbol de decision
    '''
     pass        
```

```
def predict(tree, train_dataframe)
    '''
    @param tree la estructura de datos que representa el árbol de decisión.
    @ return un dataframe con todos los datos de train_dataframe con una columna adicional que representa la probabilidad de que el income sea >50K. 
    '''
    pass
```    
Ejemplo de uso:

```
tree = train(train_dataframe)
predictions_df = predict(tree, train_dataframe)
```

In [7]:
# 

<i>[Extra credit y alumnos masters]</i>
    
5.  Mejorar la implementación de su algoritmo evitando hacer el split cuando no se logra un mínimo de Information Gain ( https://en.wikipedia.org/wiki/Information_gain_in_decision_trees   )


In [8]:
#